# Data Core Utils
> core classes & helpers

In [ ]:
#| default_exp data.core

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [ ]:
#| export

# python
from typing import List, Optional, Dict, Any, Tuple, Sequence
from abc import ABC, abstractmethod

# torch
import torch
from torchvision.transforms import transforms
from torch.utils.data import random_split, Dataset, DataLoader, Subset

import logging
logger = logging.getLogger(__name__)

## Overview
The Data Core module provides fundamental data handling, transformation, and preprocessing utilities across different machine learning domains.

## Key Features
- Flexible dataset loading and preprocessing
- Standardized data transformation interfaces
- Support for various data types and formats
- Configurable data augmentation
- Seamless integration with PyTorch and other ML frameworks

## Main Components
- Abstract base classes for datasets
- Transformation utilities
- Data loading strategies
- Preprocessing pipelines
- Compatibility with hydra configuration

## Design Principles
- Modularity: Easily extensible and composable
- Type safety: Strong typing and runtime checks
- Performance: Optimized data loading and preprocessing
- Flexibility: Support for multiple data sources and formats

## Use Cases
- Computer Vision
- Natural Language Processing
- Time Series Analysis
- Multi-modal Machine Learning

## Configuration
Supports configuration via:
- YAML files
- Hydra configuration
- Runtime parameter overrides

## Dependencies
- PyTorch
- torchvision
- hydra-core
- typing extensions


In [ ]:
#| export
class DataModule(ABC):
    def __init__(
        self,
        # data_dir: str = "~/Data/", # path to source data dir
        # train_val_split:Tuple[float, float] = [0.8, 0.2], # train val test %
        batch_size: int = 64, # size of compute batch
        num_workers: int = 0, # num_workers equal 0 means that it’s the main process that will do the data loading when needed, num_workers equal 1 is the same as any n, but you’ll only have a single worker, so it might be slow
        pin_memory: bool = False, # If you load your samples in the Dataset on CPU and would like to push it during training to the GPU, you can speed up the host to device transfer by enabling pin_memory. This lets your DataLoader allocate the samples in page-locked memory, which speeds-up the transfer
        persistent_workers: bool = False,
        shuffle: bool = True # shuffle training data
    ) -> None:
        super().__init__()
        self.save_hyperparameters() # can access inputs with self.hparams
        self.transforms = transforms.Compose([transforms.ToTensor()])
        self.train_ds: Optional[Dataset] = None
        self.val_ds: Optional[Dataset] = None
        self.test_ds: Optional[Dataset] = None

        # if sum(train_val_split) != 1.0:
        #     raise Exception('split percentages should sum up to 1.0')
    
    @abstractmethod
    def prepare_data(self) -> None:
        """Download data if needed."""
        pass
    
    @abstractmethod
    def setup(self, stage: Optional[str]=None)->None:
        """Split data into train, val, test."""
        pass

    def train_dataloader(self) -> torch.utils.data.DataLoader:
        return DataLoader(
            dataset=self.train_ds,
            batch_size=self.hparams.batch_size,
            num_workers=self.hparams.num_workers,
            pin_memory=self.hparams.pin_memory,
            shuffle=True,
            persistent_workers=self.hparams.persistent_workers
        )

    def val_dataloader(self) -> torch.utils.data.DataLoader:
        return DataLoader(
            dataset=self.val_ds,
            batch_size=self.hparams.batch_size,
            num_workers=self.hparams.num_workers,
            pin_memory=self.hparams.pin_memory,
            shuffle=False,
            persistent_workers=self.hparams.persistent_workers
        )

    def test_dataloader(self) -> torch.utils.data.DataLoader:
        return DataLoader(
            dataset=self.test_ds,
            batch_size=self.hparams.batch_size,
            num_workers=self.hparams.num_workers,
            pin_memory=self.hparams.pin_memory,
            shuffle=False,
            persistent_workers=self.hparams.persistent_workers
        )

    def teardown(self, stage: Optional[str] = None) -> None:
        """Clean up after fit or test."""
        return NotImplemented

    def state_dict(self):
        """Extra things to save to checkpoint."""
        # return {}
        return NotImplemented

    def load_state_dict(self, state_dict: Dict[str, Any]):
        """Things to do when loading checkpoint."""
        return NotImplemented

    def _sequential_split(self, dataset: Dataset, split_ratios: Sequence[float]) -> List[Subset]:
        """Split a dataset into train, validation and test sets sequentially (not randomly).
        This is useful for time series or sequence data where order matters.
        
        Args:
            dataset: Dataset to split
            split_ratios: Sequence of ratios that sum to 1.0 (e.g. [0.8, 0.1, 0.1])
            
        Returns:
            List of dataset subsets
        """
        if not isinstance(dataset, Dataset):
            raise TypeError("dataset must be a Dataset instance")
        if abs(sum(split_ratios) - 1.0) > 1e-6:
            raise ValueError("split ratios must sum to 1")
        logger.info("Split dataset into train/val/test. Keep sequence order.")
        # Calculate lengths
        total_length = len(dataset)
        lengths = [int(ratio * total_length) for ratio in split_ratios[:-1]]
        # Ensure we use all samples by assigning remainder to last split
        lengths.append(total_length - sum(lengths))
        
        # Create cumulative indices
        cum_lengths = [0] + [sum(lengths[:i+1]) for i in range(len(lengths))]
        
        # Create subsets with sequential indices
        subsets = []
        for start_idx, end_idx in zip(cum_lengths[:-1], cum_lengths[1:]):
            indices = range(start_idx, end_idx)
            subset = Subset(dataset, indices)
            subsets.append(subset)
            
        return subsets

In [ ]:
#| export

def split_train_valid_test(dataset:Dataset, splits:List[float]):
    lengths = [int(split * len(dataset)) for split in splits]
    # if rounding ends up getting rid of some datapoints and total length != length dataset
    remain = len(dataset) - sum(lengths)
    lengths[-1] += remain
    split_datasets = random_split(
                dataset=dataset,
                lengths=lengths,
                # generator=torch.Generator().manual_seed(42),
            )
    return split_datasets

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()